In [1]:
import sys

import pandas as pd
sys.path.insert(0, "..")
from dataclasses_att import *

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import extract

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dotenv import dotenv_values
import datetime


In [2]:
# Параметры подключения к БД
config = {
    **dotenv_values("../.env")
}
#pg_connection_string = f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}'
pg_connection_string = f'postgresql://{config["PG_USER"]}:{config["PG_PASSWORD"]}@{config["PG_HOST"]}:{config["PG_PORT"]}/{config["PG_DATABASE"]}'

In [3]:
engine = create_engine(pg_connection_string, echo=False)
'''
sessionmaker() создает функцию для создания сессий ORM.
autocommit=False отключает автоматическое подтверждение транзакций.
autoflush=False отключает автоматическое обновление состояния сессии после каждого запроса.
bind=engine связывает сессию с созданным движком.
'''
SessionMaker = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [4]:
sql_file = lambda x: f'{x}.sql'
csv_file = lambda x: f'result/{x}.csv'

#### 1.​ Какова доля фильмов в каждой рейтинговой категории (G, PG, PG-13, R и т.д.) в нашем ассортименте?

In [5]:
query = sqlalchemy.select(Film.rating, sqlalchemy.func.count().label("count")).\
                   group_by(Film.rating).\
                   order_by(sqlalchemy.desc("count"))
df1 = pd.read_sql(query, engine)
df1.to_csv(csv_file('df-task1'))
df1

,rating,count
0,PG-13,223
1,NC-17,210
2,R,195
3,PG,194
4,G,178


In [6]:
fig1 = px.pie(df1, values='count', names='rating', 
             title="Распределение фильмов по рейтинговым категориям",
             labels={'name': 'Рейтинговая категория', 'count': "Количество фильмов"},
             height=600, width=600)
fig1.update_traces(textposition='inside', textinfo='percent+label')
fig1.show()

#### 2.​ Какие категории фильмов чаще всего арендуются клиентами?

In [7]:
with SessionMaker() as session:
    query = session.query(Category.name, sqlalchemy.func.count(Rental.rental_id).label("count"))
    query = query.join(FilmCategory, Category.category_id == FilmCategory.category_id)
    query = query.join(Film, FilmCategory.film_id == Film.film_id)
    query = query.outerjoin(Inventory, Inventory.film_id == Film.film_id)
    query = query.outerjoin(Rental, Rental.inventory_id == Inventory.inventory_id)
    query = query.group_by(Category.name)
    query = query.order_by(sqlalchemy.desc("count"))
    df2 = pd.read_sql(query.statement, engine)
    df2.to_csv(csv_file('df-task2'))
df2

,name,count
0,Sports,1179
1,Animation,1166
2,Action,1112
3,Sci-Fi,1101
4,Family,1096
5,Drama,1060
6,Documentary,1050
7,Foreign,1033
8,Games,969
9,Children,945


In [8]:

fig2 =  px.bar(df2, x="name", y="count",
              title="Количество аренд фильмов по категориям",
              labels={'name': 'Категория', 'count': "Количество аренд"},
              text_auto='.4s',
              height=550)
fig2.update_layout(barmode='group', xaxis_tickangle=-45)
fig2.show()


#### 3.​ Какова средняя продолжительность проката (rental duration) для каждой категории фильмов?

In [9]:
with SessionMaker() as session:
    query = session.query(Category.name, sqlalchemy.func.avg(Film.rental_duration) .label("avg_rental_duration"))
    query = query.join(FilmCategory, Category.category_id == FilmCategory.category_id)
    query = query.join(Film, FilmCategory.film_id == Film.film_id)
    query = query.group_by(Category.name)
    query = query.order_by(Category.name)
    df3 = pd.read_sql(query.statement, engine)
    df3.to_csv(csv_file('df-task3'))
df3

,name,avg_rental_duration
0,Action,4.953125
1,Animation,4.893939
2,Children,5.033333
3,Classics,5.070175
4,Comedy,4.931034
5,Documentary,4.764706
6,Drama,5.080645
7,Family,5.173913
8,Foreign,5.109589
9,Games,5.065574


In [10]:
fig3 =  px.bar(df3, x="name", y="avg_rental_duration",
              title="Средняя продолжительность аренды по категориям",
              labels={'name': 'Категория', 'avg_rental_duration': "Средняя продолжительность аренды"},
              text_auto='.3s',
              height=550)
fig3.update_layout(barmode='group', xaxis_tickangle=-45)
fig3.show()


#### 4.​ Каковы тенденции в ежемесячном доходе от проката (monthly rental revenue) и продажах (sales) за прошедший год?

In [11]:
#в задании 2.4 за продажи sales берется число продаж, за доход revenue - сумма продаж
with SessionMaker() as session:
    query = session.query(extract("year", Payment.payment_date).label("year"), extract("month", Payment.payment_date).label("month"),
                          sqlalchemy.func.sum(Payment.amount).label("total_amount"), 
                          sqlalchemy.func.count().label("count"))
    query = query.select_from(Payment)
    query = query.filter(Payment.payment_date >= datetime.date(2005, 1, 1))
    query = query.filter(Payment.payment_date <= datetime.date(2005, 12, 31))
    query = query.group_by(extract("year", Payment.payment_date), extract("month", Payment.payment_date),)
    query = query.order_by(extract("year", Payment.payment_date), extract("month", Payment.payment_date),)
    df4 = pd.read_sql(query.statement, engine)
    df4.to_csv(csv_file('df-task4'))
df4.set_index(df4.apply(lambda x: f'{int(x['year'])}/{int(x['month']):02}', axis=1), inplace=True)
df4['x'] = df4['total_amount'] / df4['count']
df4

,year,month,total_amount,count,x
2005/05,2005.0,5.0,4824.43,1157,4.169775
2005/06,2005.0,6.0,9631.88,2312,4.166038
2005/07,2005.0,7.0,28373.89,6711,4.227968
2005/08,2005.0,8.0,24072.13,5687,4.232835


In [12]:
f = go.Figure()
fig42 = make_subplots(rows=1, cols=2)
fig42.update_layout(
    title_x=0.5,
    title_text="Динамика сумм и количества продаж"
)
fig42.add_trace(go.Scatter(x=df4.index, y=df4["total_amount"], name="Сумма продаж"), row=1, col=1) #secondary_y=False)
fig42.add_trace(go.Scatter(x=df4.index, y=df4["count"], name="Количество продаж"), row=1, col=2) #, secondary_y=True)
fig42.show()

#### 5.​ Как соотносятся показатели продаж в разных магазинах?

In [13]:
# в 2.5 нужно посчитать сумму продаж и сумму проданных единиц. Отобразить разницу по продажам или единицам проданного товара на графике.
with SessionMaker() as session:
    query = session.query(Store.store_id,
                          sqlalchemy.func.sum(Payment.amount).label("total_amount"), 
                          sqlalchemy.func.count().label("count"))
    query = query.select_from(Store)
    query = query.join(Customer, Store.store_id == Customer.store_id)
    query = query.join(Payment, Customer.customer_id == Payment.customer_id)
    query = query.group_by(Store.store_id)
    query = query.order_by(Store.store_id)
    df5 = pd.read_sql(query.statement, engine)
    df5['store_name'] = df5.apply(lambda x: f'Store: {int(x["store_id"])}', axis=1)
    df5.to_csv(csv_file('df-task5'))
df5

,store_id,total_amount,count,store_name
0,1,37001.52,8748,Store: 1
1,2,30414.99,7301,Store: 2


In [14]:
fig5 = go.Figure()
fig5 = make_subplots(rows=1, cols=2)
fig5.update_layout(
    title_x=0.5,
    title_text="Сравнение показателей продаж у магазинов"
)

fig5.add_trace(go.Bar(x=df5["store_name"], y=df5["total_amount"], name="Сумма продаж"), row=1, col=1)
fig5.add_trace(go.Bar(x=df5["store_name"], y=df5["count"], name="Количество продаж"), row=1, col=2)
fig5.show()

#### 6.​ Каковы средние затраты на замену (replacement_cost) фильмов в разных жанрах?

In [15]:
with SessionMaker() as session:
    query = session.query(Category.name,
                          sqlalchemy.func.avg(Film.replacement_cost).label("replacement_cost"))
    query = query.select_from(Film)
    query = query.join(FilmCategory, Film.film_id == FilmCategory.category_id)
    query = query.join(Category, FilmCategory.category_id == Category.category_id)
    query = query.group_by(Category.name)
    query = query.order_by(Category.name)
    df6 = pd.read_sql(query.statement, engine)
    df6.to_csv(csv_file('df-task6'))
df6

,name,replacement_cost
0,Action,20.99
1,Animation,12.99
2,Children,18.99
3,Classics,26.99
4,Comedy,22.99
5,Documentary,17.99
6,Drama,28.99
7,Family,15.99
8,Foreign,21.99
9,Games,24.99


In [16]:
fig6 =  px.bar(df6, x="name", y="replacement_cost",
              title="Средние затраты на замену по жанрам",
              labels={'name': 'Жанр', 'replacement_cost': "Средние затраты"},
              text_auto='$.4s',
              height=550)
fig6.update_layout(barmode='group', xaxis_tickangle=-45)
fig6.show()

#### 7.​ Какие актеры снимаются в самых разных жанрах фильмов?

In [17]:
with SessionMaker() as session:
    query = session.query(sqlalchemy.func.concat(Actor.first_name, ' ', Actor.last_name).label("actor"), 
                          sqlalchemy.func.count(Category.name.distinct()).label("count"))
    query = query.select_from(Film)
    query = query.join(FilmActor, Film.film_id == FilmActor.film_id)
    query = query.join(Actor, FilmActor.actor_id == Actor.actor_id)
    query = query.join(FilmCategory, Film.film_id == FilmCategory.category_id)
    query = query.join(Category, FilmCategory.category_id == Category.category_id)
    query = query.group_by(sqlalchemy.func.concat(Actor.first_name, ' ', Actor.last_name))
    query = query.order_by(sqlalchemy.desc("count"))
    df7 = pd.read_sql(query.statement, engine)
    df7.to_csv(csv_file('df-task7'))
df7

,actor,count
0,WARREN NOLTE,3
1,ROCK DUKAKIS,3
2,OPRAH KILMER,3
3,MINNIE ZELLWEGER,2
4,CHRISTIAN GABLE,2
...,...,...
68,GARY PHOENIX,1
69,GENE MCKELLEN,1
70,GENE WILLIS,1
71,GREGORY GOODING,1


In [18]:
fig7 =  px.bar(df7.head(5), x="actor", y="count",
              title="TOP-5 актеров с самими разнообразными жанрами",
              labels={'actor': 'Актёр', 'count': "Количество жанров"},
              text_auto='.0s',
              height=550)
fig7.update_layout(barmode='group', xaxis_tickangle=-45)
fig7.show()